# Analyze convergent clustering model performance on validation set

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns
import genetools
from IPython.display import display

In [3]:
from malid import config, logger
from malid.external import model_evaluation
from malid.datamodels import (
    combine_classification_option_names,
)
from malid.trained_model_wrappers import ConvergentClusterClassifier

# Analyze

In [5]:
for gene_locus in config.gene_loci_used:
    for target_obs_col in config.classification_targets:
        models_base_dir = ConvergentClusterClassifier._get_model_base_dir(
            gene_locus=gene_locus, target_obs_column=target_obs_col
        )  # should already exist

        output_base_dir = (
            config.paths.convergent_clusters_output_dir
            / gene_locus.name
            / combine_classification_option_names(target_obs_col)
        )  # might not yet exist
        output_base_dir.mkdir(parents=True, exist_ok=True)  # create if needed

        model_output_prefix = models_base_dir / "train_smaller_model"
        results_output_prefix = output_base_dir / "train_smaller_model"

        try:
            logger.info(
                f"{gene_locus}, {target_obs_col} from {model_output_prefix} to {results_output_prefix}"
            )

            ## Load and summarize
            experiment_set = model_evaluation.ExperimentSet.load_from_disk(
                output_prefix=model_output_prefix
            )

            # Remove global fold (we trained global fold model, but now get evaluation scores on cross-validation folds only)
            # TODO: make kdict support: del self.model_outputs[:, fold_id]
            for key in experiment_set.model_outputs[:, -1].keys():
                logger.debug(f"Removing {key} (global fold)")
                del experiment_set.model_outputs[key]

            experiment_set_global_performance = experiment_set.summarize()
            experiment_set_global_performance.export_all_models(
                func_generate_classification_report_fname=lambda model_name: f"{results_output_prefix}.classification_report.{model_name}.txt",
                func_generate_confusion_matrix_fname=lambda model_name: f"{results_output_prefix}.confusion_matrix.{model_name}.png",
                dpi=72,
            )
            combined_stats = (
                experiment_set_global_performance.get_model_comparison_stats()
            )
            combined_stats.to_csv(
                f"{results_output_prefix}.compare_model_scores.tsv",
                sep="\t",
            )
            print(gene_locus, target_obs_col)
            display(combined_stats)

            # Which p values were chosen (varies by locus, model, and fold)? How many disease-associated sequences found?
            for fold_id in config.cross_validation_fold_ids:
                for model_name in [
                    "lasso_multiclass",
                    "rf_multiclass",
                    "linearsvm_ovr",
                ]:
                    clf = ConvergentClusterClassifier(
                        fold_id=fold_id,
                        model_name=model_name,
                        fold_label_train="train_smaller",
                        gene_locus=gene_locus,
                        target_obs_column=target_obs_col,
                    )
                    p_value = clf.p_value_threshold
                    clusters = clf.cluster_centroids_with_class_specific_p_values
                    feature_names = clf.feature_names_in_
                    print(
                        f"{gene_locus}, fold {fold_id}, {target_obs_col}, {model_name}: best p value = {p_value}. Number of disease associated clusters: {(clusters[feature_names] <= p_value).sum().to_dict()}"
                    )
                print()

        except Exception as err:
            logger.exception(f"{gene_locus}, {target_obs_col} failed with error: {err}")

2023-01-07 03:53:02,592 - analyze_convergent_clustering_models.ipynb - INFO - GeneLocus.BCR, TargetObsColumnEnum.disease from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/convergent_clusters/BCR/disease/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/convergent_clusters/BCR/disease/train_smaller_model


GeneLocus.BCR TargetObsColumnEnum.disease


,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,Accuracy per fold with abstention,MCC per fold with abstention,...,au-PRC (macro OvO) per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
lasso_multiclass,0.912 +/- 0.010 (in 3 folds),0.923 +/- 0.009 (in 3 folds),0.917 +/- 0.004 (in 3 folds),0.930 +/- 0.004 (in 3 folds),0.750 +/- 0.010 (in 3 folds),0.641 +/- 0.021 (in 3 folds),0.750,0.637,0.743 +/- 0.019 (in 3 folds),0.633 +/- 0.027 (in 3 folds),...,0.931 +/- 0.005 (in 2 folds),0.743,0.628,0.009,Unknown,320,3,323,0.009288,False
linearsvm_ovr,0.912 +/- 0.004 (in 3 folds),0.922 +/- 0.004 (in 3 folds),0.919 +/- 0.007 (in 3 folds),0.931 +/- 0.006 (in 3 folds),0.754 +/- 0.012 (in 3 folds),0.647 +/- 0.017 (in 3 folds),0.754,0.643,0.749 +/- 0.005 (in 3 folds),0.642 +/- 0.008 (in 3 folds),...,0.934 +/- 0.004 (in 2 folds),0.749,0.638,0.006,Unknown,321,2,323,0.006192,False
ridge_cv,0.903 +/- 0.002 (in 3 folds),0.915 +/- 0.003 (in 3 folds),0.905 +/- 0.006 (in 3 folds),0.920 +/- 0.002 (in 3 folds),0.563 +/- 0.077 (in 3 folds),0.337 +/- 0.138 (in 3 folds),0.562,0.344,0.561 +/- 0.078 (in 3 folds),0.335 +/- 0.138 (in 3 folds),...,0.920 +/- 0.003 (in 2 folds),0.560,0.342,0.003,Unknown,322,1,323,0.003096,False
lasso_cv,0.897 +/- 0.017 (in 3 folds),0.907 +/- 0.018 (in 3 folds),0.903 +/- 0.012 (in 3 folds),0.916 +/- 0.012 (in 3 folds),0.532 +/- 0.035 (in 3 folds),0.284 +/- 0.068 (in 3 folds),0.531,0.287,0.524 +/- 0.041 (in 3 folds),0.266 +/- 0.089 (in 3 folds),...,0.921 +/- 0.000 (in 1 folds),0.523,0.268,0.015,Unknown,318,5,323,0.015480,True
rf_multiclass,0.889 +/- 0.024 (in 3 folds),0.896 +/- 0.025 (in 3 folds),0.885 +/- 0.024 (in 3 folds),0.894 +/- 0.023 (in 3 folds),0.740 +/- 0.022 (in 3 folds),0.616 +/- 0.034 (in 3 folds),0.740,0.613,0.731 +/- 0.032 (in 3 folds),0.606 +/- 0.046 (in 3 folds),...,0.888 +/- 0.000 (in 1 folds),0.731,0.602,0.012,Unknown,319,4,323,0.012384,False
xgboost,0.882 +/- 0.009 (in 3 folds),0.887 +/- 0.009 (in 3 folds),0.883 +/- 0.009 (in 3 folds),0.892 +/- 0.008 (in 3 folds),0.721 +/- 0.007 (in 3 folds),0.587 +/- 0.004 (in 3 folds),0.721,0.583,0.712 +/- 0.010 (in 3 folds),0.577 +/- 0.010 (in 3 folds),...,0.901 +/- 0.000 (in 1 folds),0.712,0.573,0.012,Unknown,319,4,323,0.012384,False
elasticnet_cv,0.878 +/- 0.023 (in 3 folds),0.891 +/- 0.019 (in 3 folds),0.891 +/- 0.020 (in 3 folds),0.906 +/- 0.017 (in 3 folds),0.590 +/- 0.129 (in 3 folds),0.370 +/- 0.219 (in 3 folds),0.589,0.386,0.587 +/- 0.133 (in 3 folds),0.368 +/- 0.223 (in 3 folds),...,0.906 +/- 0.024 (in 2 folds),0.585,0.382,0.006,Unknown,321,2,323,0.006192,False
dummy_stratified,0.536 +/- 0.032 (in 3 folds),0.543 +/- 0.036 (in 3 folds),0.529 +/- 0.020 (in 3 folds),0.535 +/- 0.024 (in 3 folds),0.378 +/- 0.047 (in 3 folds),0.066 +/- 0.060 (in 3 folds),0.377,0.064,0.366 +/- 0.040 (in 3 folds),0.064 +/- 0.055 (in 3 folds),...,NaN,0.365,0.063,0.031,Unknown,313,10,323,0.030960,False
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.467 +/- 0.016 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.466,0.000,0.452 +/- 0.010 (in 3 folds),0.028 +/- 0.020 (in 3 folds),...,NaN,0.452,0.025,0.031,Unknown,313,10,323,0.030960,True


GeneLocus.BCR, fold 0, TargetObsColumnEnum.disease, lasso_multiclass: best p value = 0.0005. Number of disease associated clusters: {'Covid19': 137, 'HIV': 151, 'Healthy/Background': 20, 'Lupus': 23}
GeneLocus.BCR, fold 0, TargetObsColumnEnum.disease, rf_multiclass: best p value = 0.005. Number of disease associated clusters: {'Covid19': 354, 'HIV': 365, 'Healthy/Background': 102, 'Lupus': 182}
GeneLocus.BCR, fold 0, TargetObsColumnEnum.disease, linearsvm_ovr: best p value = 0.001. Number of disease associated clusters: {'Covid19': 144, 'HIV': 172, 'Healthy/Background': 35, 'Lupus': 41}

GeneLocus.BCR, fold 1, TargetObsColumnEnum.disease, lasso_multiclass: best p value = 0.005. Number of disease associated clusters: {'Covid19': 301, 'HIV': 212, 'Healthy/Background': 157, 'Lupus': 291}


2023-01-07 03:53:07,843 - analyze_convergent_clustering_models.ipynb - INFO - GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/convergent_clusters/BCR/disease_all_demographics_present/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/convergent_clusters/BCR/disease_all_demographics_present/train_smaller_model


...

GeneLocus.TCR, fold 2, TargetObsColumnEnum.age_group_pediatric_healthy_only, linearsvm_ovr: best p value = 0.005. Number of disease associated clusters: {'18+': 0, 'under 18': 1171}



GeneLocus.TCR TargetObsColumnEnum.sex_healthy_only


,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,Accuracy per fold with abstention,MCC per fold with abstention,...,Abstention label global with abstention,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes,ROC-AUC (weighted OvO) per fold with abstention,ROC-AUC (macro OvO) per fold with abstention,au-PRC (weighted OvO) per fold with abstention,au-PRC (macro OvO) per fold with abstention
linearsvm_ovr,0.598 +/- 0.035 (in 3 folds),0.598 +/- 0.035 (in 3 folds),0.591 +/- 0.133 (in 3 folds),0.591 +/- 0.133 (in 3 folds),0.576 +/- 0.132 (in 3 folds),0.192 +/- 0.117 (in 3 folds),0.577,0.162,0.556 +/- 0.129 (in 3 folds),0.179 +/- 0.102 (in 3 folds),...,Unknown,111,4,115,0.034783,False,NaN,NaN,NaN,NaN
dummy_stratified,0.598 +/- 0.029 (in 3 folds),0.598 +/- 0.029 (in 3 folds),0.565 +/- 0.170 (in 3 folds),0.565 +/- 0.170 (in 3 folds),0.575 +/- 0.051 (in 3 folds),0.191 +/- 0.064 (in 3 folds),0.574,0.149,0.540 +/- 0.054 (in 3 folds),0.169 +/- 0.059 (in 3 folds),...,Unknown,108,7,115,0.060870,False,NaN,NaN,NaN,NaN
lasso_multiclass,0.594 +/- 0.052 (in 3 folds),0.594 +/- 0.052 (in 3 folds),0.602 +/- 0.132 (in 3 folds),0.602 +/- 0.132 (in 3 folds),0.543 +/- 0.085 (in 3 folds),0.160 +/- 0.070 (in 3 folds),0.541,0.078,0.514 +/- 0.068 (in 3 folds),0.145 +/- 0.050 (in 3 folds),...,Unknown,109,6,115,0.052174,False,NaN,NaN,NaN,NaN
ridge_cv,0.587 +/- 0.064 (in 3 folds),0.587 +/- 0.064 (in 3 folds),0.638 +/- 0.099 (in 3 folds),0.638 +/- 0.099 (in 3 folds),0.393 +/- 0.084 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.393,-0.234,0.384 +/- 0.092 (in 3 folds),0.046 +/- 0.041 (in 3 folds),...,Unknown,112,3,115,0.026087,False,0.567 +/- 0.000 (in 1 folds),0.567 +/- 0.000 (in 1 folds),0.630 +/- 0.000 (in 1 folds),0.630 +/- 0.000 (in 1 folds)
lasso_cv,0.533 +/- 0.044 (in 3 folds),0.533 +/- 0.044 (in 3 folds),0.551 +/- 0.186 (in 3 folds),0.551 +/- 0.186 (in 3 folds),0.523 +/- 0.015 (in 3 folds),0.176 +/- 0.111 (in 3 folds),0.523,0.045,0.495 +/- 0.015 (in 3 folds),0.144 +/- 0.117 (in 3 folds),...,Unknown,109,6,115,0.052174,False,NaN,NaN,NaN,NaN
rf_multiclass,0.524 +/- 0.035 (in 3 folds),0.524 +/- 0.035 (in 3 folds),0.540 +/- 0.143 (in 3 folds),0.540 +/- 0.143 (in 3 folds),0.528 +/- 0.078 (in 3 folds),0.148 +/- 0.066 (in 3 folds),0.548,0.101,0.401 +/- 0.199 (in 3 folds),0.112 +/- 0.088 (in 3 folds),...,Unknown,84,31,115,0.269565,False,NaN,NaN,NaN,NaN
elasticnet_cv,0.516 +/- 0.040 (in 3 folds),0.516 +/- 0.040 (in 3 folds),0.557 +/- 0.172 (in 3 folds),0.557 +/- 0.172 (in 3 folds),0.537 +/- 0.066 (in 3 folds),0.096 +/- 0.111 (in 3 folds),0.536,0.071,0.513 +/- 0.045 (in 3 folds),0.084 +/- 0.096 (in 3 folds),...,Unknown,110,5,115,0.043478,False,0.480 +/- 0.000 (in 1 folds),0.480 +/- 0.000 (in 1 folds),0.510 +/- 0.000 (in 1 folds),0.510 +/- 0.000 (in 1 folds)
xgboost,0.514 +/- 0.035 (in 3 folds),0.514 +/- 0.035 (in 3 folds),0.530 +/- 0.154 (in 3 folds),0.530 +/- 0.154 (in 3 folds),0.547 +/- 0.035 (in 3 folds),0.109 +/- 0.033 (in 3 folds),0.546,0.091,0.513 +/- 0.026 (in 3 folds),0.094 +/- 0.029 (in 3 folds),...,Unknown,108,7,115,0.060870,False,NaN,NaN,NaN,NaN
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.510 +/- 0.155 (in 3 folds),0.510 +/- 0.155 (in 3 folds),0.393 +/- 0.084 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.393,-0.234,0.384 +/- 0.092 (in 3 folds),0.046 +/- 0.041 (in 3 folds),...,Unknown,112,3,115,0.026087,False,0.500 +/- 0.000 (in 1 folds),0.500 +/- 0.000 (in 1 folds),0.486 +/- 0.000 (in 1 folds),0.486 +/- 0.000 (in 1 folds)


GeneLocus.TCR, fold 0, TargetObsColumnEnum.sex_healthy_only, lasso_multiclass: best p value = 0.01. Number of disease associated clusters: {'F': 17, 'M': 42}
GeneLocus.TCR, fold 0, TargetObsColumnEnum.sex_healthy_only, rf_multiclass: best p value = 0.005. Number of disease associated clusters: {'F': 11, 'M': 10}
GeneLocus.TCR, fold 0, TargetObsColumnEnum.sex_healthy_only, linearsvm_ovr: best p value = 0.01. Number of disease associated clusters: {'F': 17, 'M': 42}

GeneLocus.TCR, fold 1, TargetObsColumnEnum.sex_healthy_only, lasso_multiclass: best p value = 0.05. Number of disease associated clusters: {'F': 361, 'M': 431}
GeneLocus.TCR, fold 1, TargetObsColumnEnum.sex_healthy_only, rf_multiclass: best p value = 0.05. Number of disease associated clusters: {'F': 361, 'M': 431}


GeneLocus.TCR, fold 1, TargetObsColumnEnum.sex_healthy_only, linearsvm_ovr: best p value = 0.05. Number of disease associated clusters: {'F': 361, 'M': 431}

GeneLocus.TCR, fold 2, TargetObsColumnEnum.sex_healthy_only, lasso_multiclass: best p value = 0.005. Number of disease associated clusters: {'F': 3, 'M': 47}
GeneLocus.TCR, fold 2, TargetObsColumnEnum.sex_healthy_only, rf_multiclass: best p value = 0.0005. Number of disease associated clusters: {'F': 0, 'M': 4}
GeneLocus.TCR, fold 2, TargetObsColumnEnum.sex_healthy_only, linearsvm_ovr: best p value = 0.05. Number of disease associated clusters: {'F': 153, 'M': 638}

